In [1]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv3D, MaxPooling3D
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization

import os
from keras import backend as K

from sklearn.model_selection import train_test_split
import numpy as np

landmarks = [
    'PCLOrigin',
    'lateralCondyle',
    'medialCondyle',
    'medialSulcus',
    'whitesideReference'
]

Using TensorFlow backend.


In [2]:
def one_axis_femur_model(name='PCLOrigin', axis=0):
    output_path = "/code/" + name + "/"

    data_size = 15
    imgs_to_process = []
    lbls_to_process = []
    dif_arr = [(0,1,2), (0,2,1), (1,0,2), (1,2,0), (2,0,1), (2,1,0)]
    num_classes = 120

    for index in range(data_size):
        for a, b, c in dif_arr:
            imgs_to_process.append(np.load(output_path + 'images/' + '{}-{}-{}-{}.npy'.format(index, a, b, c)))
            label = np.load(output_path + 'label/' + '{}-{}-{}-{}.npy'.format(index, a, b, c))
            lbls_to_process.append(int(label[3+axis]))
            num_classes = int(label[6]) * 2

    imgs_to_process = np.array(imgs_to_process)
    lbls_to_process = np.array(lbls_to_process)

    batch_size = 15
    epochs = 200
    learning_rate = 0.001

    x_train, x_test, y_train, y_test = train_test_split(imgs_to_process, lbls_to_process, test_size=0.2)
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    img_rows = x_train.shape[1]
    img_cols = x_train.shape[2]
    colors = x_train.shape[3]


    if K.image_data_format() == 'channels_first':
        x_train = x_train.reshape(x_train.shape[0], 1,colors, img_rows, img_cols)
        x_test = x_test.reshape(x_test.shape[0], 1,colors, img_rows, img_cols)
        input_shape = (1, colors, img_rows, img_cols)
    else:
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, colors, 1)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, colors, 1)
        input_shape = (img_rows, img_cols, colors, 1)

    print(num_classes)
    # Convert class vectors to binary class matrices.
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)


    model = Sequential()
    model.add(Conv3D(3, kernel_size=(3, 3, 3), activation='relu', input_shape=input_shape))
    model.add(Conv3D(3, kernel_size=(3, 3, 3), activation='relu'))
    model.add(Conv3D(3, kernel_size=(3, 3, 3), activation='relu'))
    model.add(Conv3D(3, kernel_size=(3, 3, 3), activation='relu'))
    model.add(Conv3D(3, kernel_size=(3, 3, 3), activation='relu'))
    model.add(Conv3D(3, kernel_size=(3, 3, 3), activation='relu'))
    model.add(Conv3D(3, kernel_size=(1, 1, 1)))




    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(3,3,3)))

    model.add(Flatten())
    # Fully connected layer

    BatchNormalization()
    model.add(Dense(512))
    model.add(Activation('relu'))
    BatchNormalization()
    model.add(Dropout(0.2))
    model.add(Dense(num_classes))

    model.add(Activation('softmax'))


    sgd=SGD(lr=learning_rate)

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=sgd,
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))

    score = model.evaluate(x_test, y_test, verbose=0)
    model.save('one_axis_femur_model.h5')
    prediction = model.predict(x_test)
    print(prediction)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    print(score)
    print(y_test)


In [3]:
one_axis_femur_model('medialSulcus', 0)

x_train shape: (72, 42, 42, 42)
72 train samples
18 test samples
72 train samples
18 test samples
42
Train on 72 samples, validate on 18 samples
Epoch 1/200
72/72 [==============================] - 6s 87ms/step - loss: 19.9583 - accuracy: 0.1389 - val_loss: 3.8967 - val_accuracy: 0.2222
Epoch 2/200
72/72 [==============================] - 6s 83ms/step - loss: 3.1273 - accuracy: 0.2639 - val_loss: 3.7053 - val_accuracy: 0.2222
Epoch 3/200
72/72 [==============================] - 6s 83ms/step - loss: 2.9689 - accuracy: 0.2361 - val_loss: 3.5945 - val_accuracy: 0.2778
Epoch 4/200
72/72 [==============================] - 6s 83ms/step - loss: 2.8249 - accuracy: 0.2917 - val_loss: 3.4280 - val_accuracy: 0.2778
Epoch 5/200
72/72 [==============================] - 6s 83ms/step - loss: 2.6469 - accuracy: 0.2917 - val_loss: 3.3262 - val_accuracy: 0.2222
Epoch 6/200
72/72 [==============================] - 6s 83ms/step - loss: 2.4583 - accuracy: 0.3056 - val_loss: 3.1798 - val_accuracy: 0.2222
Ep

72/72 [==============================] - 6s 82ms/step - loss: 0.3376 - accuracy: 0.8889 - val_loss: 4.0653 - val_accuracy: 0.3333
Epoch 58/200
72/72 [==============================] - 6s 82ms/step - loss: 0.3060 - accuracy: 0.8889 - val_loss: 4.1974 - val_accuracy: 0.3333
Epoch 59/200
72/72 [==============================] - 6s 83ms/step - loss: 0.2786 - accuracy: 0.8889 - val_loss: 4.4718 - val_accuracy: 0.3333
Epoch 60/200
72/72 [==============================] - 6s 83ms/step - loss: 0.3458 - accuracy: 0.8750 - val_loss: 4.3122 - val_accuracy: 0.3333
Epoch 61/200
72/72 [==============================] - 6s 82ms/step - loss: 0.3039 - accuracy: 0.8611 - val_loss: 4.5254 - val_accuracy: 0.1111
Epoch 62/200
72/72 [==============================] - 6s 82ms/step - loss: 0.3431 - accuracy: 0.8472 - val_loss: 4.4424 - val_accuracy: 0.3333
Epoch 63/200
72/72 [==============================] - 6s 83ms/step - loss: 0.3335 - accuracy: 0.8472 - val_loss: 4.5842 - val_accuracy: 0.2778
Epoch 64/200

72/72 [==============================] - 6s 86ms/step - loss: 0.2668 - accuracy: 0.8750 - val_loss: 5.6631 - val_accuracy: 0.4444
Epoch 115/200
72/72 [==============================] - 6s 83ms/step - loss: 0.2407 - accuracy: 0.8750 - val_loss: 5.8044 - val_accuracy: 0.4444
Epoch 116/200
72/72 [==============================] - 6s 83ms/step - loss: 0.2856 - accuracy: 0.8750 - val_loss: 6.2315 - val_accuracy: 0.4444
Epoch 117/200
72/72 [==============================] - 6s 83ms/step - loss: 0.2294 - accuracy: 0.9028 - val_loss: 6.4240 - val_accuracy: 0.2778
Epoch 118/200
72/72 [==============================] - 6s 83ms/step - loss: 0.2164 - accuracy: 0.9167 - val_loss: 6.3877 - val_accuracy: 0.3333
Epoch 119/200
72/72 [==============================] - 6s 83ms/step - loss: 0.1882 - accuracy: 0.9306 - val_loss: 6.6915 - val_accuracy: 0.2778
Epoch 120/200
72/72 [==============================] - 6s 83ms/step - loss: 0.2418 - accuracy: 0.8750 - val_loss: 6.7556 - val_accuracy: 0.4444
Epoch 

Epoch 171/200
72/72 [==============================] - 6s 82ms/step - loss: 0.1999 - accuracy: 0.9306 - val_loss: 7.3403 - val_accuracy: 0.2778
Epoch 172/200
72/72 [==============================] - 6s 82ms/step - loss: 0.2079 - accuracy: 0.9167 - val_loss: 7.5231 - val_accuracy: 0.2778
Epoch 173/200
72/72 [==============================] - 6s 82ms/step - loss: 0.1590 - accuracy: 0.9306 - val_loss: 7.6894 - val_accuracy: 0.2778
Epoch 174/200
72/72 [==============================] - 6s 82ms/step - loss: 0.1988 - accuracy: 0.9028 - val_loss: 7.8193 - val_accuracy: 0.2778
Epoch 175/200
72/72 [==============================] - 6s 82ms/step - loss: 0.2215 - accuracy: 0.9306 - val_loss: 7.8497 - val_accuracy: 0.3333
Epoch 176/200
72/72 [==============================] - 6s 82ms/step - loss: 0.2131 - accuracy: 0.9167 - val_loss: 7.8157 - val_accuracy: 0.3333
Epoch 177/200
72/72 [==============================] - 6s 83ms/step - loss: 0.1726 - accuracy: 0.9306 - val_loss: 8.0441 - val_accuracy: